<a href="https://colab.research.google.com/github/AchrafAsh/gnn-linear-receptive-fields/blob/main/bottlenecks_of_gnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import os, sys
from google.colab import drive
drive.mount('/content/mnt')
nb_path = '/content/notebooks'
os.symlink('/content/mnt/My Drive/Colab Notebooks', nb_path)
sys.path.insert(0, nb_path)  # or append(nb_path)

Drive already mounted at /content/mnt; to attempt to forcibly remount, call drive.mount("/content/mnt", force_remount=True).


In [6]:
import torch
import torch_geometric as tg
import networkx

In [7]:
import os.path as osp
from torch_geometric.datasets import Planetoid, TUDataset

'/content'

In [8]:
path = osp.join(os.getcwd(), 'data', 'Cora')
dataset = Planetoid(path, 'Cora')
# path = osp.join(os.getcwd(), 'data', 'ENZYMES')
# dataset = TUDataset(path, name='ENZYMES')

Processing...
Done!


In [32]:
G = dataset[0]
print(G.x)
print(G.edge_index)
print(G.y)
print(G['train_mask'])
print(G)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([[   0,    0,    0,  ..., 2707, 2707, 2707],
        [ 633, 1862, 2582,  ...,  598, 1473, 2706]])
tensor([3, 4, 4,  ..., 3, 3, 3])
tensor([ True,  True,  True,  ..., False, False, False])


Data(edge_index=[2, 10556], test_mask=[2708], train_mask=[2708], val_mask=[2708], x=[2708, 1433], y=[2708])

In [92]:
# MAD - Mean Average Distance (global)
def mean_average_distance(G):
    '''
    Computes cosine similarity between each pair of node representations
    '''
    A = torch_geometric.utils.to_dense_adj(G.edge_index)
    # D = torch.zeros(G.num_nodes, G.num_nodes)

    MAD_num = 0
    MAD_den = 0

    for i in range(G.num_nodes):
        D_avg_num = 0
        D_avg_den = 0
        for j in range(i):
            cos_sim = 0
            if A[0, i, j]:
                cos_sim = 1 - torch.dot(G.x[i,:], G.x[j,:]) / (torch.norm(G.x[i, :]) * torch.norm(G.x[j, :]) + 1e-16)
            D_avg_num += cos_sim
            D_avg_den += 1 if cos_sim > 0 else 0
        
        if D_avg_den:
            D_avg = D_avg_num / D_avg_den
            MAD_num += D_avg_num
            MAD_den += 1 if D_avg > 0 else 0

    MAD = MAD_num / MAD_den
    return MAD

In [93]:
mean_average_distance(G)

tensor(2.1652)

In [ ]:
def mean_average_distance_gap(g):
    '''
    Computes MADGap: MAD_remote - MAD_neighbors
    '''
    